# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

from azureml.widgets import RunDetails                                        # needed to access the RunDetails widget in section "Run Details", below. 
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

import joblib

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
experiment_name = 'heart-failure-experiment-hyperd'
project_folder = './capstone-project'

experiment = Experiment(ws, experiment_name)
experiment

# starting an interactive logging session, as recommended in Azure documentation 'how-to-log-view-metrics'
run=experiment.start_logging()

In [ ]:
# Create compute cluster and choose a name for it
cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

## Dataset

### Overview

I will use [Kaggle](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) "Heart Failure Prediction dataset".  
This dataset is related to a study that  focused on survival analysis of 299 heart failure patients who were admitted to Institute   
of Cardiology and Allied hospital Faisalabad-Pakistan during April-December (2015).   
All the patients were aged 40 years or above, having left ventricular systolic dysfunction.  

The dataset contains the following 12 clinical features, plus one target feature ("death event"):  
A data analysis report is available onmy github repo, [here](https://github.com/JCForszp/nd00333-capstone/blob/master/Datasets/heart%20failure%20report.html)

**Clinical features:**
- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- high blood pressure: if the patient has hypertension (boolean)
- creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days)

**Target feature:**
- [target] death event: if the patient deceased during the follow-up period (boolean)

We are dealing here with a classification task, i.e trying to predict the outcome of the follow-up period based on the given clinical features.


In [ ]:
found = False
key = "JCF-heart-failure-dataset"
description_text = "Kaggle Heart Failure Prediction dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/JCForszp/nd00333-capstone/blob/master/Datasets/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

## Hyperdrive Configuration

For this part of the project, I chose a Logistic Regression model that fits well binary classification problems.  
Features are weakly correlated and the presence of categorical variables (anemia, diabetes, sex) is supported by this algorithm.  
So, LR seems well suited. 

The aim here is to fine-tune the model hyper-parameters using Azure HyperDrive. 
HyperDrive configuration will be split below into 3 sections:
1. early termination policy
2. creation of the estimator and of the different parameters that will be used during the training
3. the hyper drive configuration run in itself

### 1. early_termination_policy
> Regarding early termination of poorly performing runs, I used the BanditPolicy.  
> The BanditPolicy defines a slack factor (defined here to 0.1).  
> All runs that fall outside the slack factor with respect to the best performing run will be terminated, saving time and budget.

### 2. estimator and parameters sampling
**estimator**
> We will use a Logistic Regression and use 'accuracy'.  
> AuC would also have been an option.  

**Hyperparameter space**  
> I chose the RandomParameterSampling, mainly for speed reason, as the usual alternative, GridParameterSampling, would have triggered   
> an exhaustive search over the complete space, for a gain that proved to be relatively small at the end.  
> Also, GridParameterSampling only allows discrete values, while random sampling is more open as it allows also the use of continuous values.  
> Finally, RandomParameterSampling supports early termination of low-performance runs.  
> For those three reasons, and within the given context of this analysis, RandomParameterSampling appeared as the best option.  

### 3. HyperDrive run configuration  
> This configuration object aggregates the settings defined for the policy, the choice of estimator and the hyper-parameters space definition.  
> We define here also the primary metric used ('accuracy') that we want to maximize (**primary_metric_goal**).  
> **max_total_runs** sets a limit of the maximum number of runs that can be created. We set it here to 100.  
> **max_concurrent_runs** is aligned to the compute target available resources available (4).  



In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1)

#TODO: Create the different params that you will be using during training
param_sampling = = RandomParameterSampling(
                        {
                            "--C": uniform(0.01 , 1.99),
                            "--max_iter": choice(range(50,150,10))
                        }
                        )

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=os.path.join("./"), compute_target=compute_cluster_target,entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(estimator=estimator, 
                                    hyperparameter_sampling=param_sampling, 
                                    policy=early_termination_policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=100,
                                    max_concurrent_runs=4)


In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
hdr.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run=hyperdrive_run.get_best_run_by_primary_metric()
best_run

In [ ]:
print(f"Best run metrics: {best_run.get_metrics()}.")

In [ ]:
#TODO: Save the best model
best_run.register_model(model_name = "best_run_hyperdrive.pkl", model_path = './outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service